<div style="float:left;"><img src="logo.png" width="500"/></div>

# Link Prediction

This demo will focus on more advanced topics, specifically using node embeddings for link prediction. The notebook requires the *DGL*, *PyTorch*, and *scikit-learn* Python packages.

For installation instruction for DGL, see:
https://www.dgl.ai/pages/start.html

In network analysis **link prediction** is a widely-applied task which involves predicting the existence of a link between two nodes in a network. Examples of link prediction include predicting friendship links among users in an online social network platform or predicting co-authorship links in a scientific citation network.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools, random
import pandas as pd
import numpy as np
import scipy.sparse as sp

# imports for DGL
import dgl
import dgl.data
from dgl.nn import SAGEConv
import dgl.function as fn

# imports from scikit-learn
from sklearn.metrics import roc_auc_score

# display settings
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size': 14})

Set up number generation:

In [ ]:
def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
set_all_seeds(100)

## Data Preparation

Load the "Cora" scientific network dataset, which is included as part of DGL. 

Note that we could also create a NetworkX graph and convert it for use with DGL, using the *dgl.from_networkx()* function.

In [ ]:
dataset = dgl.data.CoraGraphDataset()
# check the structure of the dataset
g = dataset[0]

Now we prepare the training and testing data. 

In link prediction tasks, it is usual to treat existing edges in the graph as **positive examples**, and non-existent edges as **negative examples**. 

In this case we will use 10% of the edges for positive examples in the test set, and leave the rest for the training set.

In [ ]:
# get the edges
u, v = g.edges()
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)

# split the existing edge set for training and testing
# these are the "positive edges"
test_frac = 0.1
test_size = int(len(eids) * test_frac)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# find all the negative edges and split them for training and testing
# these are the "negative edges"
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)
neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

When training, we need to hide the edges in the test set from the original graph:

In [ ]:
train_g = dgl.remove_edges(g, eids[:test_size])

In [ ]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

## Model Preparation

We will use the **GraphSAGE** algorithm for generating node embeddings from our network.

For more information on GraphSAGE, see: https://snap.stanford.edu/graphsage

In [ ]:
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        # add GraphSAGE layers to our neural network architecture
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

To use the node embedding vectors, we will use a **dot product** measure to compute the similarlity between two vectors:

In [ ]:
class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

We will also define a loss function, which is the standard *binary cross entropy loss*:

In [ ]:
def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

## Training Phase

Create the model and optimizer:

In [ ]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 16)
pred = DotPredictor()
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

Apply training for the specified number of epochs:

In [ ]:
max_epochs = 100
loss_scores = {}
# run each epoch...
for e in range(1, max_epochs+1):
    # forward step
    h = model(train_g, train_g.ndata['feat'])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)
    
    # backward step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # store the score
    loss_scores[e] = float(loss)
    if e % 20 == 0:
        print('Epoch %d/%d' % (e, max_epochs))        

Plot the trajectory of the loss function:

In [ ]:
ax = pd.Series(loss_scores).plot(figsize=(10, 6), zorder=3)
ax.set_xlabel("Training Epoch")
ax.set_ylabel("Loss")
ax.yaxis.grid()
ax.set_xlim(1, max_epochs)
ax.set_ylim(0);

## Evaluation Phase

Using our trained model and our test date, compute the **Area-Under-the-Curve (AUC)**, which is a measure of the ability of a classifier to distinguish between the classes.

In [ ]:
def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [ ]:
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC = %.4f' % compute_auc(pos_score, neg_score))